In [27]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xbg
from sklearn.grid_search import GridSearchCV
from time import time
from operator import itemgetter

In [2]:
infile = pd.read_csv("../DataDir/sant-train.csv")

infile.columns.values
infile.TARGET.value_counts()

colcard = list()

for i in xrange(len(infile.columns)):
    colcard.append(len(infile.iloc[:,i].value_counts()))

In [3]:
colnz = infile.astype(bool).sum(axis=0)
len(colnz[colnz>50])
#infile['saldo_medio_var13_medio_hace3'].describe()

229

In [5]:
trainfeat = infile.drop(['TARGET'], axis=1)
trainlab = infile['TARGET']

X_train, X_test, y_train, y_test = cross_validation.train_test_split(trainfeat, trainlab, test_size=0.30, random_state=777)

X_train.shape, y_train.shape

#X_test.shape, y_test.shape

((53214, 370), (53214,))

In [11]:
clf = RandomForestClassifier(n_estimators=500,n_jobs=4,)
newfit = clf.fit(X_train,y_train)
#clf.score(X_test, y_test, scoring="roc_auc")

In [30]:
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [31]:
#Next we can do a grid search to find the best parameters for the RFC
param_grid = {"max_depth": [5,3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"],
              "n_estimators": [100,500,1000]}

param_grid_sm = {"max_depth": [5,3, None],
              "criterion": ["gini", "entropy"],
              "n_estimators": [100,500,1000]}

grid_search = GridSearchCV(clf, param_grid=param_grid_sm, n_jobs=4)
start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.grid_scores_)))
report(grid_search.grid_scores_)

print grid_search.best_params_
bestcv = grid_search.best_params_
#{'n_estimators': 100, 'criterion': 'gini', 'max_depth': 5}

/Users/jeff/.local/lib/python2.7/site-packages/sklearn/utils/__init__.py:157: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/jeff/.local/lib/python2.7/site-packages/sklearn/utils/__init__.py:157: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/jeff/.local/lib/python2.7/site-packages/sklearn/utils/__init__.py:157: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/jeff/.local/lib/python2.7/site-packages/sklearn/utils/__init__.py:157: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/jeff/.local/lib/python2.7/site-packages/sklearn/utils/__init__.py:157: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/jeff/.local/lib/python2.7/site-packages/sklearn/utils/__init__.py:157: DataConversionWarning: Copying input dataframe for slicing.
  DataConversionWarning)
/Users/jeff/.local/lib

GridSearchCV took 711.28 seconds for 18 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.960 (std: 0.000)
Parameters: {'n_estimators': 100, 'criterion': 'gini', 'max_depth': 5}

Model with rank: 2
Mean validation score: 0.960 (std: 0.000)
Parameters: {'n_estimators': 500, 'criterion': 'gini', 'max_depth': 5}

Model with rank: 3
Mean validation score: 0.960 (std: 0.000)
Parameters: {'n_estimators': 1000, 'criterion': 'gini', 'max_depth': 5}

{'n_estimators': 100, 'criterion': 'gini', 'max_depth': 5}


In [16]:
pred_prob=newfit.predict_proba(X_train)
roc_auc_score(y_train,pred_prob[:,1])


1.0

In [18]:
pred_test=newfit.predict_proba(X_test)
roc_auc_score(y_test,pred_test[:,1])

0.7888148912960733

In [7]:
scores = cross_validation.cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=3)

In [8]:
scores


array([ 0.78785387,  0.78989648,  0.80078597])

In [ ]:
#let's try normalization of parameters

#remove cols with <50 values


# #psuedo code:
# for i in n_cols
#     1. swap out -99999999 and other odd vals
#     2. scale vars

#try booleans
# for i in n_cols
#     1. swap out -99999999 and other odd vals
#     2. if val > 0, mark as 1

#look up text classif algos/sparse
infile.columns